https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

# Trivial Case: len(input) == len(output)

https://raw.githubusercontent.com/fchollet/keras/master/examples/addition_rnn.py

In [8]:
# -*- coding: utf-8 -*-
'''An implementation of sequence to sequence learning for performing addition

Input: "535+61"
Output: "596"
Padding is handled by using a repeated sentinel character (space)

Input may optionally be inverted, shown to increase performance in many tasks in:
"Learning to Execute"
http://arxiv.org/abs/1410.4615
and
"Sequence to Sequence Learning with Neural Networks"
http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf
Theoretically it introduces shorter term dependencies between source and target.

Two digits inverted:
+ One layer LSTM (128 HN), 5k training examples = 99% train/test accuracy in 55 epochs

Three digits inverted:
+ One layer LSTM (128 HN), 50k training examples = 99% train/test accuracy in 100 epochs

Four digits inverted:
+ One layer LSTM (128 HN), 400k training examples = 99% train/test accuracy in 20 epochs

Five digits inverted:
+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs
'''

from __future__ import print_function
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range


class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.

        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.

        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
INVERT = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS + 1 - len(ans))
    if INVERT:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

# Try replacing GRU, or SimpleRNN.
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

print('Build model...')
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last hidden state of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 200):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if INVERT else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

Generating data...
Total addition questions: 50000
Vectorization...
Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_4 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_4 (TimeDist (None, 4, 12)             1548      
_________________________________________________________________
activation_4 (Activation)    (None, 4, 12)             0         
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_____________________________

45000/45000 [==============================] - 14s 314us/step - loss: 0.4241 - acc: 0.8641 - val_loss: 0.3650 - val_acc: 0.8882
Q 996+6   T 1002 ☑ 1002
Q 688+7   T 695  ☑ 695 
Q 55+84   T 139  ☑ 139 
Q 402+363 T 765  ☒ 764 
Q 975+456 T 1431 ☒ 1430
Q 428+6   T 434  ☑ 434 
Q 664+7   T 671  ☑ 671 
Q 22+35   T 57   ☒ 67  
Q 73+151  T 224  ☑ 224 
Q 553+53  T 606  ☑ 606 

--------------------------------------------------
Iteration 14
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 13s 285us/step - loss: 0.2988 - acc: 0.9240 - val_loss: 0.2706 - val_acc: 0.9292
Q 17+122  T 139  ☑ 139 
Q 234+46  T 280  ☑ 280 
Q 60+437  T 497  ☑ 497 
Q 0+41    T 41   ☒ 40  
Q 9+481   T 490  ☑ 490 
Q 466+336 T 802  ☑ 802 
Q 441+2   T 443  ☑ 443 
Q 42+56   T 98   ☒ 99  
Q 739+71  T 810  ☑ 810 
Q 26+333  T 359  ☒ 358 

--------------------------------------------------
Iteration 15
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

45000/45000 [==============================] - 14s 303us/step - loss: 0.0400 - acc: 0.9899 - val_loss: 0.0179 - val_acc: 0.9976
Q 221+516 T 737  ☑ 737 
Q 406+33  T 439  ☑ 439 
Q 144+28  T 172  ☑ 172 
Q 75+620  T 695  ☑ 695 
Q 801+19  T 820  ☑ 820 
Q 415+829 T 1244 ☑ 1244
Q 52+762  T 814  ☑ 814 
Q 77+80   T 157  ☑ 157 
Q 60+99   T 159  ☑ 159 
Q 924+1   T 925  ☑ 925 

--------------------------------------------------
Iteration 28
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 15s 333us/step - loss: 0.0119 - acc: 0.9993 - val_loss: 0.0147 - val_acc: 0.9982
Q 71+623  T 694  ☑ 694 
Q 45+69   T 114  ☑ 114 
Q 139+3   T 142  ☑ 142 
Q 221+310 T 531  ☑ 531 
Q 124+252 T 376  ☑ 376 
Q 238+36  T 274  ☑ 274 
Q 835+12  T 847  ☑ 847 
Q 5+680   T 685  ☑ 685 
Q 660+658 T 1318 ☑ 1318
Q 206+42  T 248  ☑ 248 

--------------------------------------------------
Iteration 29
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

45000/45000 [==============================] - 13s 294us/step - loss: 0.0031 - acc: 0.9999 - val_loss: 0.0053 - val_acc: 0.9989
Q 99+764  T 863  ☑ 863 
Q 917+35  T 952  ☑ 952 
Q 203+92  T 295  ☑ 295 
Q 1+905   T 906  ☑ 906 
Q 711+22  T 733  ☑ 733 
Q 75+48   T 123  ☑ 123 
Q 70+52   T 122  ☑ 122 
Q 4+606   T 610  ☑ 610 
Q 56+973  T 1029 ☑ 1029
Q 702+4   T 706  ☑ 706 

--------------------------------------------------
Iteration 42
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 14s 318us/step - loss: 0.0027 - acc: 0.9999 - val_loss: 0.0054 - val_acc: 0.9989
Q 325+40  T 365  ☑ 365 
Q 5+26    T 31   ☑ 31  
Q 980+10  T 990  ☑ 990 
Q 49+526  T 575  ☑ 575 
Q 33+182  T 215  ☑ 215 
Q 1+83    T 84   ☑ 84  
Q 726+255 T 981  ☑ 981 
Q 722+688 T 1410 ☑ 1410
Q 352+80  T 432  ☑ 432 
Q 776+958 T 1734 ☑ 1734

--------------------------------------------------
Iteration 43
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

18816/45000 [===========>..................] - ETA: 8s - loss: 0.0687 - acc: 0.9773

KeyboardInterrupt: 

# General Case: canonical sequence-to-sequence

simaple machine translation